# Prediction check for two level classifier


## 1. Set up

In [ ]:
import os
import json
import glob
import scipy
import numpy as np

# import preprocess func from processor module
from processor import DataSet

from keras.models import model_from_json
from keras.models import model_from_yaml
from keras.utils import np_utils
from keras.preprocessing import image

import pandas as pd
import glob

In [ ]:
# set root as ../
import sys
sys.path.append("../")

In [ ]:
BASE_MODEL_DIR = "./trained_model/inceptionv3/"
BASE_NAME = ""
CODE_EXTRACTOR=""

In [ ]:
BASE_CODE_NAME = "code_" + BASE_NAME

In [ ]:
DATA_DIR = ""

BASE_DATA_DIR= os.path.normpath(os.path.join(DATA_DIR, ''))

# Use this for get catkeys.
VALID_DATA_DIR = os.path.normpath(os.path.join(BASE_DATA_DIR, "valid"))

In [ ]:
CATSTORE_PATH = ""

# Get Model from S3

In [ ]:
REGION_NAME = "ap-northeast-1"
DEFAULT_OUTPUT = "text"

DIST_DATA_DIR = os.path.normpath(BASE_DATA_DIR)

os.environ['AWS_DEFAULT_REGION'] = REGION_NAME
os.environ['AWS_DEFAULT_OUTPUT'] = DEFAULT_OUTPUT

!aws s3 sync 

# Target data setup

In [ ]:
from one_vs_all import ModelBinder
from codeextractor import DNNCodeExtractor

def ModelBinder_create(base_model_name = BASE_CODE_NAME, basedir = BASE_MODEL_DIR, extractor_path=CODE_EXTRACTOR):
    return ModelBinder(base_model_name, basedir, DNNCodeExtractor.create_from(extractor_path))

In [ ]:
CHECK_DATA_DIR = ''

In [ ]:
CHECK_DATA_PATHS = list(glob.glob(os.path.normpath("{}/*.jpg").format(CHECK_DATA_DIR)))

In [ ]:
CHECK_DATA_DIR1 = ''
CHECK_DATA_DIR2 = ''
CHECK_DATA_SPAGHETTI=''

In [ ]:
CHECK_DATA_PATHS1 = list(glob.glob(os.path.normpath("{}/*.jpg").format(CHECK_DATA_DIR1)))
CHECK_DATA_PATHS2 = list(glob.glob(os.path.normpath("{}/*.jpg").format(CHECK_DATA_DIR2)))
CHECK_DATA_PATHS_SPAGHETTI = list(glob.glob(os.path.normpath("{}/*.jpg").format(CHECK_DATA_SPAGHETTI)))


In [ ]:
import random, itertools

In [ ]:
# CHECK_DATA_PATHS = list(itertools.chain(random.sample(CHECK_DATA_PATHS1, 5000), CHECK_DATA_PATHS2))
CHECK_DATA_PATHS = list(itertools.chain(CHECK_DATA_PATHS1[0:5000], CHECK_DATA_PATHS2[0:(5000-len(CHECK_DATA_PATHS_SPAGHETTI))], CHECK_DATA_PATHS_SPAGHETTI))

In [ ]:
len(CHECK_DATA_PATHS)

## TwoLevelClassifier

In [ ]:
# 54 category, loosen 5 categories.
catproc = [
]

In [ ]:
h2binder = ModelBinder_create()
h1binder = ModelBinder.dup_from(h2binder)

In [ ]:
from one_vs_all import TwoLevelClassifier

In [ ]:
two = TwoLevelClassifier(catproc, h2binder, h1binder)

In [ ]:
two.load_all()

In [ ]:
res = two.predict_files_df(CHECK_DATA_PATHS)

In [ ]:
len(res)

In [ ]:
filtered = res[res['category'] != -1]

In [ ]:
len(filtered)

In [ ]:
218/1995

In [ ]:
from category import VirtualCategories
vc = VirtualCategories.from_file(CATSTORE_PATH)


In [ ]:
from visualize import plot_image_list

In [ ]:
def plot_result(df, cat, vc):
    print(vc.name(cat))
    plot_image_list(df[df['category'] == cat]["filepaths"].values[0:40])

In [ ]:
def plot_result2(cat):
    plot_result(filtered, cat, vc)

In [ ]:
# plot_result2('11_13_54_120_130_58_86_96')

### 54 cat

In [ ]:
# plot_result2('43_80')

In [ ]:
# plot_result2('20')

In [ ]:
# salad, NG
# plot_result2('57_64')

In [ ]:
# plot_result2('93')

In [ ]:
# plot_result2('45')

In [ ]:
# plot_result2('4')

In [ ]:
# plot_result2('65_81')

In [ ]:
# plot_result2('71')

In [ ]:
# plot_result2('11_13_54_120_130_58_86_96')

In [ ]:
# plot_result2('129')

In [ ]:
# plot_result2('176')

In [ ]:
# plot_result2('16')

In [ ]:
# plot_result2('167')

In [ ]:
# plot_result2('114')

In [ ]:
# pan, NG
# plot_result2('110')

In [ ]:
# plot_result2('147')

In [ ]:
# plot_result2('38')

In [ ]:
# plot_result2('76')

In [ ]:
# plot_result2('8')

In [ ]:
# plot_result2('160')

In [ ]:
# plot_result2('39')

### Other misc final result check

In [ ]:
# plot_result(filtered, '93', vc)

In [ ]:
# plot_result(filtered, '104', vc)

In [ ]:
# plot_result(filtered, '16', vc)

In [ ]:
plot_result(filtered, '83', vc)

In [ ]:
plot_result(filtered, '100', vc)

In [ ]:
# plot_result(filtered, '39', vc)

In [ ]:
# plot_result(filtered, '102', vc)

In [ ]:
# plot_result(filtered, '10', vc)

In [ ]:
plot_result(filtered, '20', vc)

In [ ]:
# plot_result(filtered, '47_73', vc)

In [ ]:
# plot_result(filtered, '71', vc)

In [ ]:
# plot_result(filtered, '160', vc)

In [ ]:
# plot_result(filtered, '11_13_54_120_130_58_86_96', vc)

In [ ]:
# plot_result2('107')

In [ ]:
plot_result2('63')

In [ ]:
# plot_result2('72_108_156_162')

In [ ]:
cats = {cat for cat in filtered['category'].values}

In [ ]:
cats

### Result evaluation

In [ ]:
TWO_PRED_SCORE_PATH=""

In [ ]:
df = two._df

In [ ]:
df['filepaths'] = CHECK_DATA_PATHS

In [ ]:
df.to_pickle(TWO_PRED_SCORE_PATH)

In [ ]:
df = pd.read_pickle(TWO_PRED_SCORE_PATH)

In [ ]:
flicker_df = df

In [ ]:
from visualize import plot_image_list

In [ ]:
def plot_cat(df, cat, thrld, origin=0):
    filtered = df[df[cat] > thrld].sort_values(by=cat, ascending=False)
    for i in range(0, 50, 5):
        print(",".join(["{0:.4f}".format(v) for v in filtered[cat].values[origin+i:(origin+5+i)]]))
    plot_image_list(filtered["filepaths"].values[origin:(origin+50)])

In [ ]:
def plot_cat2(cat, origin=0):
    print(vc.name(cat))
    plot_cat(df, cat, 0.1, origin)

In [ ]:
plot_cat2('57_64')

In [ ]:
plot_cat2('93')

In [ ]:
plot_cat2('45')

In [ ]:
plot_cat2('4')

In [ ]:
plot_cat2('65_81')

In [ ]:
plot_cat2('71')

In [ ]:
plot_cat2('11_13_54_120_130_58_86_96')

In [ ]:
plot_cat2('129')

In [ ]:
plot_cat2('176')

In [ ]:
plot_cat2('16')

In [ ]:
plot_cat2('167')

In [ ]:
plot_cat2('114')

In [ ]:
plot_cat2('110')

In [ ]:
plot_cat2('147')

In [ ]:
plot_cat2('38')

In [ ]:
plot_cat2('76')

In [ ]:
plot_cat2('8')

In [ ]:
plot_cat2('160')

In [ ]:
# plot_cat(df, "93", 0.94)

In [ ]:
vc.name('100')

In [ ]:
# plot_cat(df, "100", 0.72)

In [ ]:
vc.name('47_73')

In [ ]:
# plot_cat(df, "47_73", 0.72)

In [ ]:
vc.name('71')

In [ ]:
# plot_cat(df, "71", 0.72, 0)

In [ ]:
# plot_cat(df, "11_13_54_120_130_58_86_96", 0.72, 0)

In [ ]:
# plot_cat(df, "160", 0.72, 0)

In [ ]:
vc.name('150')

In [ ]:
# plot_cat(df, "150", 0.72)

In [ ]:
vc.name('109')

In [ ]:
# plot_cat(df, "109", 0.6)

In [ ]:
vc.name('10')

In [ ]:
# plot_cat(df, "10", 0.97)

In [ ]:
vc.name("100")

In [ ]:
# plot_cat(df, "100", 0.94)

In [ ]:
# plot_cat(df, "102", 0.84)

In [ ]:
# plot_cat(df, "106", 0.8)

### Threshold history

In [ ]:
# loosen, 
catproc = [
]

### Threshold pick

In [ ]:
DIST_THRESHOLD=1.5

#created by classifier_similarity.ipynb
VALID_DF_PATH=""
VALID_DISTANCE_DICT_PATH=""

In [ ]:
scoredf = pd.read_pickle(VALID_DF_PATH)

In [ ]:
distdf = pd.read_pickle(VALID_DISTANCE_DICT_PATH)

In [ ]:
from functools import reduce

In [ ]:
def calc_raw_scores(targetkey, thrshold):
    similarkeyset = set(distdf[targetkey][distdf[targetkey] >= DIST_THRESHOLD].index)
    
    allfalse = np.zeros(len(scoredf), dtype=bool)
    tp_indices = reduce((lambda x, y: x | y), [allfalse] + [scoredf['label'] == key for key in similarkeyset])
    TP = scoredf[tp_indices]
    TN = scoredf[~tp_indices]
    FP = TN[TN[targetkey] > thrshold]
    FN = TP[TP[targetkey] <= thrshold]
    return {"TP": len(TP), "TN":len(TN), "FP":len(FP), "FN":len(FN)}

In [ ]:
def best_f1_threshold(targetkey):
    thrds = [val*0.01 for val in range(70, 100)] + [0.99, 0.995, 0.999, 0.9995, 0.9999]
    raw_scores = [calc_raw_scores(targetkey, thrd) for thrd in thrds]
    
    scoretups = [(dic["TP"]/(dic["TP"]+dic["FP"]), dic["TP"]/(dic["TP"]+dic["FN"]), 2*dic["TP"]/(2*dic["TP"]+dic["FP"]+dic["FN"])) for dic in raw_scores]
    f1scores = pd.DataFrame({"precision":[tup[0] for tup in scoretups], "recall":[tup[1] for tup in scoretups], "f1":[tup[2] for tup in scoretups]})
    f1scores.index = thrds
    return f1scores["f1"].idxmax()

In [ ]:
[(cat, best_f1_threshold(cat)) for cat in catproc.keys()] 

In [ ]:
catproc = {
}

In [ ]:
targetkey='100'

In [ ]:
thrds = [0.99, 0.995, 0.999, 0.9995]

In [ ]:
raw_scores = [calc_raw_scores(targetkey, thrd) for thrd in thrds]

In [ ]:
scoretups = [(dic["TP"]/(dic["TP"]+dic["FP"]), dic["TP"]/(dic["TP"]+dic["FN"]), 2*dic["TP"]/(2*dic["TP"]+dic["FP"]+dic["FN"])) for dic in raw_scores]

In [ ]:
f1scores = pd.DataFrame({"precision":[tup[0] for tup in scoretups], "recall":[tup[1] for tup in scoretups], "f1":[tup[2] for tup in scoretups]})

In [ ]:
f1scores.index = thrds

In [ ]:
f1scores["f1"].idxmax()

In [ ]:
f1scores.plot()

In [ ]:
from modelutils import dir2filedict
catkeys =  sorted(list(dir2filedict(VALID_DATA_DIR).keys()))

In [ ]:
catkeys[135], catkeys[148]

In [ ]:
#targetkey='60'
targetkey='72'


In [ ]:
similarkeyset = set(distdf[targetkey][distdf[targetkey] >= DIST_THRESHOLD].index)

In [ ]:
similarkeyset

In [ ]:
thrdsorg = list(range(70, 100))

In [ ]:
thrds = [val*0.01 for val in thrdsorg]

In [ ]:
raw_scores = [calc_scores('106', thrd) for thrd in thrds]

In [ ]:
scoretups = [(dic["TP"]/(dic["TP"]+dic["FP"]), dic["TP"]/(dic["TP"]+dic["FN"]), 2*dic["TP"]/(2*dic["TP"]+dic["FP"]+dic["FN"])) for dic in raw_scores]

In [ ]:
f1scores = pd.DataFrame({"precision":[tup[0] for tup in scoretups], "recall":[tup[1] for tup in scoretups], "f1":[tup[2] for tup in scoretups]})

In [ ]:
f1scores.index = thrdsorg

In [ ]:
%matplotlib inline

In [ ]:
f1scores["f1"].max()

In [ ]:
f1scores["f1"].idxmax()

In [ ]:
f1scores["f1"][90]

In [ ]:
f1scores.plot()

In [ ]:
targetkey = '106'

In [ ]:
similarkeyset = set(distdf[targetkey][distdf[targetkey] >= DIST_THRESHOLD].index)

In [ ]:
sum(scoredf['label'] == targetkey)

In [ ]:
tmp = list(similarkeyset)

In [ ]:
sum(scoredf['label'] == tmp[0])

In [ ]:
sum((scoredf['label'] == tmp[0]) |( scoredf['label'] == tmp[1]))

In [ ]:
res = np.zeros(len(scoredf), dtype=bool)

In [ ]:
sum(res | (scoredf['label'] == tmp[0]))

In [ ]:
list(range(80, 100))

In [ ]:
calc_scores('106', 0.97)

In [ ]:
allfalse = np.zeros(len(scoredf), dtype=bool)

In [ ]:
tp_indices = reduce((lambda x, y: x | y), [allfalse] + [scoredf['label'] == key for key in similarkeyset])

In [ ]:
TP = scoredf[tp_indices]

In [ ]:
len(TP)

In [ ]:
TN = scoredf[~tp_indices]

In [ ]:
FP = TN[TN[targetkey] > thrshold]

In [ ]:
FN = TP[TP[targetkey] <= thrshold]

In [ ]:
len(TP), len(FP), len(TN), len(FN)

In [ ]:
# f1scores = f1scores.drop('thresholds', 1)

In [ ]:
scoredf[0:4]['filepaths'].values

In [ ]:
thrshold = 0.97

In [ ]:
filtered = scoredf[scoredf[targetkey] > thrshold]

In [ ]:
filtered

### Try and error

In [ ]:
res = two.predict_arrs(arrs)

In [ ]:
resdf = pd.DataFrame({"filepaths":testfiles, "category":[tup[0] for tup in res], "label":[tup[1] for tup in res]})

In [ ]:
resdf[resdf['category'] != -1]

In [ ]:
cat106 = resdf[resdf["category"] == '106']

In [ ]:
cat106

In [ ]:
sum(two._df['100'] > 0.97)

In [ ]:
filtered = two._df[two._df['100'] > 0.97]

In [ ]:
tmp =filtered['100'].values[0:5]

In [ ]:
",".join([str(v) for v in tmp])

In [ ]:
print(",".join(["{0:.3f}".format(v) for v in filtered['100'].values[0:5]]))
print(",".join(["{0:.3f}".format(v) for v in filtered['100'].values[5:5+5]]))
print(",".join(["{0:.3f}".format(v) for v in filtered['100'].values[10:10+5]]))

### Small test code

In [ ]:
testfiles = CHECK_DATA_PATHS[0:100]

In [ ]:
from processor import DataSet
ds = DataSet()
arrs = ds.files_to_dataset(testfiles)

In [ ]:
catkeys = two._catkeys

In [ ]:
["h1_"+cat for cat in catkeys]